In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set(style='whitegrid')
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [ ]:
import os
import pandas as pd

# Set folder path where all Blinkit CSV files are stored
folder_path = r"D:/PythonFiles/project/blinkit"

# Dictionary to store all dataframes
dataframes = {}

# Manually define all expected file names (remove .csv to use as keys)
csv_files = [
    "blinkit_customer_feedback",
    "blinkit_customers",
    "blinkit_delivery_performance",
    "blinkit_inventory",
    "blinkit_inventoryNew",
    "blinkit_marketing_performance",
    "blinkit_order_items",
    "blinkit_orders",
    "blinkit_products",
    "blinkit_customer_feedback - Copy"
]

# Load each file into the dictionary
for file_name in csv_files:
    full_path = os.path.join(folder_path, file_name + ".csv")
    try:
        dataframes[file_name] = pd.read_csv(full_path)
        print(f"✅ Loaded: {file_name} ({dataframes[file_name].shape[0]} rows, {dataframes[file_name].shape[1]} columns)")
    except Exception as e:
        print(f"❌ Failed to load {file_name}: {e}")

In [ ]:
# ✅ Assign individual DataFrames to variables
orders = dataframes['blinkit_orders']
delivery = dataframes['blinkit_delivery_performance']
customers = dataframes['blinkit_customers']

In [ ]:
# ✅ Step 2: Convert Dates
orders['order_date'] = pd.to_datetime(orders['order_date'])
delivery['promised_time'] = pd.to_datetime(delivery['promised_time'])
delivery['actual_time'] = pd.to_datetime(delivery['actual_time'])

In [ ]:
merged = pd.merge(orders, delivery, on='order_id', how='inner')
full_data = pd.merge(merged, customers, on='customer_id', how='left')

In [ ]:
print("Order Total Description:")
print(orders['order_total'].describe())

print("\nDelivery Time (Minutes) Description:")
print(full_data['delivery_time_minutes'].describe())

print("\nCustomer Segment Distribution:")
print(customers['customer_segment'].value_counts())

In [ ]:
orders = pd.read_csv("D:/PythonFiles/blinkit/blinkit_orders.csv")
orders.head()

In [ ]:
orders.info()

In [ ]:
orders.describe()

In [ ]:
orders.isnull().sum()

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(orders['order_total'], kde=True, color='g', bins=30)
plt.title("Distribution of Order Values")
plt.xlabel("Order Total (₹)")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

In [ ]:
#  Step 6: Visualization – Customer Segment Count
plt.figure(figsize=(6, 4))
sns.countplot(data=customers, x="customer_segment", palette="viridis")
plt.title("Customer Segment Distribution")
plt.xlabel("Segment")
plt.ylabel("Number of Customers")
plt.tight_layout()
plt.show()

In [ ]:
orders['order_date'] = pd.to_datetime(orders['order_date'])

In [ ]:
# Monthly revenue
monthly_revenue = orders.resample('M', on='order_date')['order_total'].sum()

plt.figure(figsize=(10, 5))
monthly_revenue.plot()
plt.title("Monthly Revenue")
plt.ylabel("Revenue (₹)")
plt.xlabel("Month")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Load Feedback Data
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt

feedback = pd.read_csv("D:/PythonFiles/blinkit/blinkit_customer_feedback.csv")

# Remove nulls and join all feedback
text = ' '.join(feedback['feedback_text'].dropna().astype(str))

# Generate WordCloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Common Words in Customer Feedback')
plt.tight_layout()
plt.show()

In [ ]:
# Merge feedback and orders
merged = pd.merge(feedback, orders, on='order_id', how='left')

# Merge delivery info
merged = pd.merge(merged, delivery[['order_id', 'promised_time', 'actual_time']], on='order_id', how='left')

# Convert to datetime
merged['promised_time'] = pd.to_datetime(merged['promised_time'])
merged['actual_time'] = pd.to_datetime(merged['actual_time'])

# Calculate delivery time in minutes
merged['delivery_time_minutes'] = (merged['actual_time'] - merged['promised_time']).dt.total_seconds() / 60

In [ ]:
sentiment_analysis = merged.groupby('sentiment').agg({
    'order_total': 'mean',
    'delivery_time_minutes': 'mean'
}).reset_index()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Set better & darker color palette
colors = ['#2c7bb6', '#abd9e9', '#d7191c']  # Dark Blue, Light Blue, Red

plt.figure(figsize=(6, 4))
barplot = sns.barplot(data=sentiment_analysis, x='sentiment', y='order_total', palette=colors)

# Add value labels on top of each bar
for index, row in sentiment_analysis.iterrows():
    barplot.text(index, row.order_total + 50, f"{row.order_total:.0f}", color='black', ha="center", fontweight='bold',size =9)

plt.title("Average Order Value by Sentiment", fontsize=12)
plt.ylabel("Avg Order Total (₹)", fontsize=12)
plt.xlabel("Feedback Sentiment", fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
sns.barplot(data=sentiment_analysis, x='sentiment', y='delivery_time_minutes', palette='coolwarm')
plt.title("Average Delivery Delay by Sentiment")
plt.ylabel("Avg Delivery Time (minutes)")
plt.xlabel("Feedback Sentiment")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Custom dark color palette (Positive = Green, Neutral = Black, Negative = Red)
custom_colors = {
    'Positive': '#008000',   # Dark Green
    'Neutral': '#333333',    # Dark Grey (Black-ish)
    'Negative': '#880808'    # Firebrick Red
}

# Map sentiment to color
palette = [custom_colors[sentiment] for sentiment in sentiment_analysis['sentiment']]

plt.figure(figsize=(8, 5))
barplot = sns.barplot(
    data=sentiment_analysis,
    x='sentiment',
    y='delivery_time_minutes',
    palette=palette
)

plt.title("Average Delivery Delay by Sentiment", fontsize=14)
plt.ylabel("Avg Delivery Time (minutes)", fontsize=12)
plt.xlabel("Feedback Sentiment", fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Make a copy of delivery data
model_data = delivery[['delivery_time_minutes', 'distance_km', 'promised_time', 'delivery_partner_id']].copy()

# Drop nulls
model_data.dropna(inplace=True)

# Extract hour and weekday from promised_time
model_data['promised_hour'] = pd.to_datetime(model_data['promised_time']).dt.hour
model_data['promised_weekday'] = pd.to_datetime(model_data['promised_time']).dt.weekday

In [ ]:
# Convert delivery_partner_id to categorical code
model_data['delivery_partner_id'] = model_data['delivery_partner_id'].astype('category').cat.codes

In [ ]:
from sklearn.model_selection import train_test_split

X = model_data[['distance_km', 'promised_hour', 'promised_weekday', 'delivery_partner_id']]  # features
y = model_data['delivery_time_minutes']  # target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

model = LinearRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("📉 Mean Squared Error:", round(mse, 2))
print("📈 R² Score:", round(r2, 2))

In [ ]:
plt.figure(figsize=(10, 8))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.6, color='green')
plt.xlabel("Actual Delivery Time")
plt.ylabel("Predicted Delivery Time")
plt.title("Actual vs Predicted Delivery Time")
plt.tight_layout()
plt.show()

In [ ]:
coef_df = pd.DataFrame({'Feature': X.columns, 'Coefficient': model.coef_})
print(coef_df)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Predict on test set
y_pred = model.predict(X_test)

# Accuracy metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("📉 Mean Squared Error (MSE):", round(mse, 2))
print("📈 R² Score:", round(r2, 2))

In [ ]:
# Re-create model_data (same as you did earlier)
model_data = delivery[['delivery_time_minutes', 'distance_km', 'promised_time', 'delivery_partner_id']].copy()

# Drop nulls
model_data.dropna(inplace=True)

# Extract hour and weekday from promised_time
model_data['promised_time'] = pd.to_datetime(model_data['promised_time'])
model_data['promised_hour'] = model_data['promised_time'].dt.hour
model_data['promised_weekday'] = model_data['promised_time'].dt.weekday

# Convert delivery_partner_id to category codes
model_data['delivery_partner_id'] = model_data['delivery_partner_id'].astype('category').cat.codes

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# ✅ Step 1: Prepare Data (already done by you earlier)
X = model_data[['distance_km', 'promised_hour', 'promised_weekday', 'delivery_partner_id']]
y = model_data['delivery_time_minutes']

# ✅ Step 2: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Step 3: Train Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# ✅ Step 4: Make Predictions
y_pred_rf = rf_model.predict(X_test)

# ✅ Step 5: Evaluate Model
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("🌲 Random Forest Results:")
print("📉 Mean Squared Error:", round(mse_rf, 2))
print("📈 R² Score:", round(r2_rf, 2))

In [ ]:
# Show coefficients of each feature
import pandas as pd

coef_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
}).sort_values(by='Coefficient', key=abs, ascending=False)

coef_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8,5))
sns.barplot(data=coef_df, x='Coefficient', y='Feature', palette='Greens')
plt.title("Impact of Features on Delivery Time (Linear Regression)")
plt.xlabel("Coefficient Value")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

In [ ]:
customers = pd.read_csv("D:/PythonFiles/blinkit/blinkit_customers.csv")
customers.head()

In [ ]:
clustering_data = customers[['total_orders', 'avg_order_value']].copy()
clustering_data.dropna(inplace=True)

In [ ]:
customers.head()

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(clustering_data)

In [ ]:
wcss = []  # Within-cluster sum of squares
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(scaled_data)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(6,4))
plt.plot(range(1, 11), wcss, marker='o')
plt.title('Elbow Method for Optimal K')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('WCSS')
plt.tight_layout()
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42)
customers['cluster'] = kmeans.fit_predict(scaled_data)

In [ ]:
plt.figure(figsize=(8, 5))
sns.scatterplot(data=customers, x='total_orders', y='avg_order_value', hue='cluster', palette='Set2')
plt.title("Customer Segmentation Clusters")
plt.xlabel("Total Orders")
plt.ylabel("Average Order Value")
plt.legend(title='Cluster')
plt.tight_layout()
plt.show()

In [ ]:
customers.groupby('cluster')[['total_orders', 'avg_order_value']].mean()

In [ ]:
# Step 1: Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings("ignore")

In [ ]:
plt.figure(figsize=(10, 5))
sns.lineplot(data=monthly_sales, x=monthly_sales.index, y='order_total', marker='o')
plt.title("Monthly Sales (Order Total)")
plt.ylabel("Total Revenue (₹)")
plt.xlabel("Month")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Step 2: Convert order_date to datetime format (very important for time series)
orders['order_date'] = pd.to_datetime(orders['order_date'])

In [ ]:
# Step 3: Group by Month and Get Monthly Sales
monthly_sales = orders.groupby(pd.Grouper(key='order_date', freq='M')).sum(numeric_only=True)[['order_total']]

# Optional: View first few rows
monthly_sales.head()

In [ ]:
# Step 4: Plot Historical Sales
plt.figure(figsize=(10,5))
plt.plot(monthly_sales.index, monthly_sales['order_total'], marker='o', linestyle='-', color='blue')
plt.title("📈 Monthly Sales Trend ")
plt.xlabel("Month")
plt.ylabel("Total Sales (₹)")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Step 5: Build SARIMAX Time Series Model
model = SARIMAX(monthly_sales['order_total'], 
                order=(1, 2, 1), 
                seasonal_order=(1, 1, 1, 12))

results = model.fit(disp=False)

In [ ]:
# Step 6: Forecast for Next 3 Months (1 Quarter)
forecast = results.get_forecast(steps=3)
forecast_df = forecast.conf_int()
forecast_df['Predicted Sales'] = forecast.predicted_mean
forecast_df.index = pd.date_range(start=monthly_sales.index[-1] + pd.DateOffset(months=1), 
                                  periods=3, freq='MS')

forecast_df

In [ ]:
import matplotlib.dates as mdates

plt.figure(figsize=(12, 6))

# Plot historical sales
plt.plot(monthly_sales.index, monthly_sales['order_total'], label='Historical Sales', color='blue', marker='o')

# Plot forecast
plt.plot(forecast_df.index, forecast_df['Predicted Sales'], label='Forecast (Next 3 Months)', color='green', marker='o', linestyle='--')

# Confidence interval shaded area
plt.fill_between(forecast_df.index,
                 forecast_df['lower order_total'],
                 forecast_df['upper order_total'],
                 color='lightgreen', alpha=0.4, label='Confidence Interval')

# Highlight forecast region background
plt.axvspan(forecast_df.index[0], forecast_df.index[-1], color='gray', alpha=0.1)

# Format x-axis as month/year
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b-%Y'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))

plt.title("Sales Forecast vs Historical Sales")
plt.xlabel("Month")
plt.ylabel("Sales (₹)")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Re-create model_data (same as you did earlier)
model_data = delivery[['delivery_time_minutes', 'distance_km', 'promised_time', 'delivery_partner_id']].copy()

# Drop nulls
model_data.dropna(inplace=True)

# Extract hour and weekday from promised_time
model_data['promised_time'] = pd.to_datetime(model_data['promised_time'])
model_data['promised_hour'] = model_data['promised_time'].dt.hour
model_data['promised_weekday'] = model_data['promised_time'].dt.weekday

# Convert delivery_partner_id to category codes
model_data['delivery_partner_id'] = model_data['delivery_partner_id'].astype('category').cat.codes